# Research Paper Recommender

### :Review article recommender using PubMed API and Key word exraction from article titles using TF-IDF

During my research period, I have been switching my research topic very frequently, so I know how much the pain involves whenever they had to restart studying on the new field. Thus, I am currently developing the educational platform for junior researchers who just started research, using paper and keyword recommender system.

**Keyword recommendation:** Problem of current research article search engine is providing the keywords based on the keywords provided by article author, and mostly the one-word synonyms. I developed the keyword recommender using texts from titles and abstracts. Proper usage of ngram and TF-IDF provided intuitive keywords which gives information what to look up for the study.

**Paper recommendation:** Best way to learn new field is reading a good review article written by leading scientist in the field. Problem of finding the article by citation number is this will be highly biased towards the date of publication. First article published in the field gets most citation. This recommender provides list of review articles of most active researchers, based on publication number and citation recently, and displays most recently reviews by analyzing PubMed database. 

Currently working on the building a database using searched keywords and develop abstract analyzer

# Importing Packages and Functions

In [1]:
from Bio import Entrez
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import itertools

pd.set_option('display.max_colwidth', 1000)

In [2]:
def search(query):
    Entrez.email = 'sxxize@gmail.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='1000',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

In [3]:
def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'sxxize@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [4]:
def Author_list(papers):
    paper_author_lst=[i['MedlineCitation']['Article']['AuthorList']\
                      for i in papers['PubmedArticle']]
    dfs=[pd.DataFrame(paper_author_lst[i]) for i in range(len(paper_author_lst))]
    names_dfs=pd.concat(dfs, axis=0, sort=True )
    author_count_df=names_dfs[['ForeName', 'LastName']]\
                    .groupby(['ForeName', 'LastName']).size()\
                    .reset_index(name='count').sort_values(by='count', ascending=False)
    top=author_count_df.head(10)
    google_url='https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q='
    name=top['ForeName']+' '+top['LastName']
    result=top.reset_index(drop=True).join(pd.DataFrame({'Google Scholar':[google_url+i for i in name.str.replace(' ', '+')+'+review&oq=']}))
    
    return result

In [23]:
def key_from_papers(papers):
    fetch_key_word_papers=[i['MedlineCitation']['KeywordList'] for i in papers['PubmedArticle']]
    lst_key_papers=list(itertools.chain.from_iterable(list(itertools.chain.from_iterable(fetch_key_word_papers))))
    key_from_paper=pd.DataFrame({'key word from papers':lst_key_papers})
    key_word=key_from_paper['key word from papers'].str.lower()
    return key_word

In [6]:
def title_key(papers):
    titles=[i['MedlineCitation']['Article']['ArticleTitle'].lower()\
            .replace(',','').replace('.','').replace(':', '').replace('?','')\
            .replace('<sub>', '').replace('</sub>','').replace('<sup>','').replace('</sup>','')\
            .replace('<i>','').replace('</i>','')\
            .replace(search_word.lower(),'') for i in papers['PubmedArticle']]
    tfidf=TfidfVectorizer(ngram_range=(2,2),stop_words='english')
    X=tfidf.fit_transform(titles)
    tfidf_df=pd.DataFrame(X.todense(), columns=sorted(tfidf.vocabulary_))
    key_rank=tfidf_df.sum().sort_values(ascending=False)
    
    return key_rank


In [7]:
def Abstract_key(papers):
    abstract_key=[]
    for i in papers['PubmedArticle']:
        try:
            abstract_key.append(i['MedlineCitation']['Article']['Abstract']['AbstractText'][0].lower()\
            .replace(',','').replace('.','').replace(':', '').replace('?','')\
            .replace('<sub>', '').replace('</sub>','').replace('<sup>','').replace('</sup>','')\
            .replace('<i>','').replace('</i>','').replace(search_word.lower(),''))
        except:
            continue
    tfidf=TfidfVectorizer(ngram_range=(2,2),stop_words='english')
    X=tfidf.fit_transform(abstract_key)
    tfidf_df=pd.DataFrame(X.todense(), columns=sorted(tfidf.vocabulary_))
    key_rank=tfidf_df.sum().sort_values(ascending=False)
    
    return key_rank

In [8]:
search_word='bioactive'

In [9]:
results = search(search_word)
id_list = results['IdList']
papers = fetch_details(id_list)

# Author List

In [10]:
result=Author_list(papers)
result

,ForeName,LastName,count,Google Scholar
0,Aldo R,Boccaccini,34,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Aldo+R+Boccaccini+review&oq=
1,Francesco,Baino,11,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Francesco+Baino+review&oq=
2,Mohamed N,Rahaman,10,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Mohamed+N+Rahaman+review&oq=
3,Jiang,Chang,10,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Jiang+Chang+review&oq=
4,Julian R,Jones,9,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Julian+R+Jones+review&oq=
5,Wenhai,Huang,9,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Wenhai+Huang+review&oq=
6,Chengtie,Wu,9,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Chengtie+Wu+review&oq=
7,Hui,Wang,7,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Hui+Wang+review&oq=
8,Bo,Lei,7,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Bo+Lei+review&oq=
9,Saeid,Kargozar,7,https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Saeid+Kargozar+review&oq=


In [11]:
result['Google Scholar']

0    https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Aldo+R+Boccaccini+review&oq=
1      https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Francesco+Baino+review&oq=
2    https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Mohamed+N+Rahaman+review&oq=
3          https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Jiang+Chang+review&oq=
4       https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Julian+R+Jones+review&oq=
5         https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Wenhai+Huang+review&oq=
6          https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Chengtie+Wu+review&oq=
7             https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Hui+Wang+review&oq=
8               https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Bo+Lei+review&oq=
9       https://scholar.google.co.kr/scholar?hl=ko&as_sdt=0%2C5&q=Saeid+Kargozar+review&oq=
Name: Google Scholar, dtype: object

# Key Word List

In [24]:
key_paper_lst=key_from_papers(papers)

In [27]:
key_paper_lst.value_counts().head(20)

bioactive glass            141
bioactive compounds        114
bioactive peptides          54
bone regeneration           33
bioactivity                 31
antioxidant activity        23
scaffold                    23
bioactive                   22
bioactive glasses           21
angiogenesis                19
strontium                   19
antioxidant                 19
tissue engineering          18
bioactive peptide           18
cytotoxicity                17
scaffolds                   16
bone tissue engineering     16
osteogenesis                16
mechanical properties       15
hydroxyapatite              15
Name: key word from papers, dtype: int64

# Key Words from Title

In [14]:
key_title=title_key(papers)
key_title[:20]

tissue engineering       7.418839
bone tissue              5.608003
bone regeneration        5.454114
mesoporous glass         5.142716
antioxidant activity     5.029007
mass spectrometry        4.581099
derived peptides         4.420087
glass nanoparticles      4.402085
glass scaffolds          4.288539
compounds antioxidant    4.107077
extraction compounds     3.715036
scaffolds bone           3.529002
stem cells               3.463278
liquid chromatography    3.339055
natural products         3.333138
glass based              3.149904
containing glass         3.146186
mechanical properties    2.952051
ultrasound assisted      2.790998
composite scaffolds      2.787624
dtype: float64

# Key Words from Abstract

In [15]:
key_abstract=Abstract_key(papers)

In [16]:
key_abstract[:20]

tissue engineering       4.688455
antioxidant activity     4.455011
bone regeneration        4.331385
bone tissue              4.081456
mechanical properties    3.919917
body fluid               3.479476
anti inflammatory        3.348498
simulated body           3.313052
present study            3.296120
aim study                3.155519
stem cells               3.103526
sol gel                  3.087326
glass nanoparticles      2.968362
bone defects             2.843119
fatty acids              2.754742
natural products         2.712150
liquid chromatography    2.693476
mass spectrometry        2.618745
electron microscopy      2.532576
scanning electron        2.527362
dtype: float64

LDA
word Vec
pyldavis

pyldavis

textrank
lexrank